# MLflow

MLflow is an open source platform to manage the ML lifecycle, including experimentation, reproducibility and deployment. It currently offers three components:
    
- *MLflow Tracking*: Record and query experiments: code, data, config, and results.
- *MLflow Projects*: Packaging format for reproducible runs on any platform.
- *MLflow Models*: General format for sending models to diverse deployment tools.

    
    

## MLflow Tracking
The MLflow Tracking component is an API and UI for logging parameters, code versions, metrics, and output files when running your machine learning code and for later visualizing the results. MLflow Tracking lets you log and query experiments using Python, REST, R API, and Java API APIs.

### Concepts
MLflow Tracking is organized around the concept of runs, which are executions of some piece of data science code. Each run records the following information:

#### Code Version
Git commit hash used for the run, if it was run from an `MLflow Project`.

#### Start & End Time
Start and end time of the run

#### Source
Name of the file to launch the run, or the project name and entry point for the run if run from an `MLflow Project`.

#### Parameters
Key-value input parameters of your choice. Both keys and values are strings.

#### Metrics
Key-value metrics, where the value is numeric. Each metric can be updated throughout the course of the run (for example, to track how your model’s loss function is converging), and MLflow records and lets you visualize the metric’s full history.

#### Artifacts
Output files in any format. For example, you can record images (for example, PNGs), models (for example, a pickled scikit-learn model), and data files (for example, a Parquet file) as artifacts.
You can record runs using MLflow Python, R, Java, and REST APIs from anywhere you run your code. For example, you can record them in a standalone program, on a remote cloud machine, or in an interactive notebook. If you record runs in an MLflow Project, MLflow remembers the project URI and source version.


You can optionally organize runs into experiments, which group together runs for a specific task. You can create an experiment using the mlflow experiments CLI, with `mlflow.create_experiment()`, or using the corresponding REST parameters. The MLflow API and UI let you create and search for experiments.

## Install MLflow SDK

In [ ]:
!pip3 install mlflow==1.3.0 keras boto3 --user

## Install MLFlow Tracking Server

Open mlflow/mlflow.yaml and update `<YOUR_S3_BUCKET_NAME>` to an existing bucket name and make sure you have `aws-secret` in kubeflow namespace. 

In [ ]:
# Deploy Mlflow tracking server
# Since you are in `anonymous` namespace, you may not have permission to create components in kubeflow namespace.
# Instead, please copy file to your local envrionment and this command.

!kubectl apply -f mlflow/mlflow.yaml

## Example

MLflow Tracking is an API and UI for logging parameters, code versions, metrics and output files when running your machine learning code to later visualize them. With a few simple lines of code, you can track parameters, metrics, and artifacts:


```py
import mlflow

# Log parameters (key-value pairs)
mlflow.log_param("num_dimensions", 8)
mlflow.log_param("regularization", 0.1)

# Log a metric; metrics can be updated throughout the run
mlflow.log_metric("accuracy", 0.1)
...
mlflow.log_metric("accuracy", 0.45)

# Log artifacts (output files)
mlflow.log_artifact("roc.png")
mlflow.log_artifact("model.pkl")

```

You can use MLflow Tracking in any environment (for example, a standalone script or a notebook) to log results to local files or to a server, then compare multiple runs. Using the web UI, you can view and compare the output of multiple runs. Teams can also use the tools to compare results from different users:

![mlflow-web-ui](mlflow/mlflow-web-ui.png)

## Full Example

In [ ]:
from __future__ import print_function

import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import os
import subprocess
import argparse
import time

import mlflow
import mlflow.keras


# Reduce spam logs from s3 client
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'

def preprocessing():
  fashion_mnist = keras.datasets.fashion_mnist
  (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

  # scale the values to 0.0 to 1.0
  train_images = train_images / 255.0
  test_images = test_images / 255.0

  # reshape for feeding into the model
  train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
  test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

  class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

  print('\ntrain_images.shape: {}, of {}'.format(train_images.shape, train_images.dtype))
  print('test_images.shape: {}, of {}'.format(test_images.shape, test_images.dtype))

  return train_images, train_labels, test_images, test_labels

def train(train_images, train_labels, epochs, model_summary_path):
  if model_summary_path:
    logdir=model_summary_path # + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

  model = keras.Sequential([
    keras.layers.Conv2D(input_shape=(28,28,1), filters=8, kernel_size=3,
                        strides=2, activation='relu', name='Conv1'),
    keras.layers.Flatten(),
    keras.layers.Dense(10, activation=tf.nn.softmax, name='Softmax')
  ])
  model.summary()

  model.compile(optimizer=tf.train.AdamOptimizer(),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
                )
  if model_summary_path:
    model.fit(train_images, train_labels, epochs=epochs, batch_size=64, callbacks=[tensorboard_callback])
  else:
    model.fit(train_images, train_labels, epochs=epochs, batch_size=64)

  mlflow.log_param('batch_size', 64)

  return model

def eval(model, test_images, test_labels):
  test_loss, test_acc = model.evaluate(test_images, test_labels)
  print('\nTest accuracy: {}, test loss {}'.format(test_acc, test_loss))
  mlflow.log_metric("accuracy", test_acc)
  mlflow.log_metric("loss", test_loss)

def export_model(model, model_export_path):
  version = 1
  export_path = os.path.join(model_export_path, str(version))

  tf.saved_model.simple_save(
    keras.backend.get_session(),
    export_path,
    inputs={'input_image': model.input},
    outputs={t.name:t for t in model.outputs})

  print('\nSaved model: {}'.format(export_path))


def main(argv=None):
  parser = argparse.ArgumentParser(description='Fashion MNIST Tensorflow Example')
  parser.add_argument('--model_export_path', type=str, help='Model export path')
  parser.add_argument('--model_summary_path', type=str,  help='Model summry files for Tensorboard visualization')
  parser.add_argument('--epochs', type=int, default=5, help='Training epochs')
  args = parser.parse_args(args=['--epochs=10'])

  # File Based Tracking URI. Use NFS in this case
  # users_home = '/tmp/shjiaxin'
  # experiment_base_path = '%s/experiments' % users_home
  # tracking_uri='file://%s' % experiment_base_path
  
  # Remote Tracking Server URI. Use kubernetes Service.
  # Make sure you have installed MLflow
  tracking_uri = "http://mlflow-tracking-server.kubeflow.svc.cluster.local:5000"
  mlflow.set_tracking_uri(tracking_uri)

  experiment_name = 'mlflow'
  mlflow.set_experiment(experiment_name)

  with mlflow.start_run() as run:
    start_time = time.time()
    train_images, train_labels, test_images, test_labels = preprocessing()
    model = train(train_images, train_labels, args.epochs, args.model_summary_path)
    eval(model, test_images, test_labels)

    mlflow.log_param('epochs', args.epochs)

    if args.model_export_path:
      export_model(model, args.model_export_path)

    # Use MLFlow fashion to persist model
    mlflow.keras.log_model(model, 'model_keras')

    # Measure running time
    duration_in_seconds = time.time() - start_time
    print("This model took", duration_in_seconds, "seconds to train and test.")
    mlflow.log_metric("time_duration", duration_in_seconds)

    
if __name__ == "__main__":
  main()

## Compare performance

In your terminal, run `kubectl port-forward svc/mlflow-tracking-server -n kubeflow 5000:5000`. Then you can visit `localhost:5000` in your browser.